# Домашнее задание 0

## URL:

- https://ru.kinorium.com/name/138/?order=premier_date&show_viewed=1&popular=1
- https://www.film.ru/person/leonardo-dikaprio


# Домашнее задание 1

In [178]:
import requests
import bs4
import openpyxl
import json
import codecs

wb=openpyxl.Workbook()
sheet=wb.active

def main():
    url1 = 'https://ru.kinorium.com/name/138/?order=premier_date&show_viewed=1&popular=1' 
    url2 = 'https://www.film.ru/person/leonardo-dikaprio'
    content1 = getData(url1)
    data1 = parseData1(content1)
    saveData("./data1.json",data1)
    
    content2 = getData(url2)
    data2 = parseData2(content2)
    saveData("./data2.json",data2)
    
def getData(url):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'}
    response = requests.get(url,headers=headers)
    status_code = response.status_code
    content = bs4.BeautifulSoup(response.content.decode("utf-8"), "lxml")
    return content
    
def saveData(path,data):
    wb=openpyxl.Workbook()
    sheet=wb.active
    f = codecs.open(path , 'w' , 'utf-8' )
    f.write(json.dumps(data,ensure_ascii=False))
    f.close()
def getmidstring(html, start_str, end):
    start = html.find(start_str)
    if start >= 0:
        start += len(start_str)
        end = html.find(end)
        if end >= 0:
            return html[start:end].strip() 
        
def parseData1(content):
    obj = {}
    element = content.find("table", class_="infotable").find_all("tr")
    for item in element:
        legend = item.find(class_="legend").text
        data = item.find(class_="data").text
        obj[legend] = data
    element = content.find_all("a", class_="movieAwards__item")
    obj["rewards"] = []
    for item in element:
        rewards = {}
        title = item.find(class_="title").text
        wins = item.find(class_="wins")
        if(wins):
            wins = wins.text
        else:
            wins = ""
        noms = item.find(class_="noms")
        if(noms):
            wins = wins +" " + noms.text
        rewards[title] = wins
        obj["rewards"].append(rewards)
    element = content.find_all("div", class_="filmList__item-content")
    obj["работает"] = []
    count = 0;
    for item in element:
        count += 1;
        title = item.find(class_="filmList__movie-name").find("i").text.strip()
        engname = item.find(class_="filmList__small-text") #Оригинальное название
        if engname != None:
            engname = engname.text.strip()
        else:
            engname = ""
        details = str(item.find("p",class_="filmList__extra-info"))
        details = getmidstring(details,'<p class="filmList__extra-info">',"<br/>")
        director = item.find(class_="filmList__extra-info-director")
        director = director.text + director.find("a").text.strip()
        film = {
            "id": count,
            "title":title,
            "original name": engname,
            "extra":details,
            "director":director
        }
        obj["работает"].append(film)
    return obj
def parseData2(content):
    obj = {}
    element = content.find(class_="movies-center-table").find_all("div")
    for item in element:
        data = item.find_all('strong')
        obj[data[0].text] = data[1].text.replace("\n", "").replace("  ", "")
    obj["работает"] = []
    count = 0;
    element = content.find_all(class_="filmography")
    for item in element:
        con = item.find_all("div")[1]
        count += 1;
        title = con.find("a").text
        name = con.find("strong").text.replace("\n", "")
        year = con.text.replace(name, "").replace(title, "")
        film = {
            "id": count,
            "title":title,
            "original name": name,
            "year": year,
        }
        obj["работает"].append(film)
    return obj
main()

# Домашнее задание 2


In [177]:
import json

def saveData(path,data):
    wb=openpyxl.Workbook()
    sheet=wb.active
    f = codecs.open(path , 'w' , 'utf-8' )
    f.write(json.dumps(data,ensure_ascii=False))
    f.close()

def readJSON(path):
    with open(path,'r',encoding='utf8')as fp:
        json_data = json.load(fp)
    return json_data

json_data1 = readJSON('./data1.json')
json_data2 = readJSON('./data2.json')

for key,values in  json_data1.items():
    if key not in json_data2:
        json_data2[key] = values

work1 = json_data1["работает"]
work2 = json_data2["работает"]

obj = {}
for item in work1:
    obj[item["title"]] = item

for item in work2:
    title = item["title"]
    if title in obj:
        obj2 = obj[title]
        for key,values in obj2.items():
            if key not in item:
                item[key] = values
                
json_data2["работает"] = work2

saveData('./mash.json',json_data2)